In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import nltk

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
nltk.download('stopwords', 'all')

C:\Users\44758\AppData\Local\Temp\ipykernel_25308\1752672697.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


[nltk_data] Downloading package stopwords to all...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Download stopwords (only needed once)
nltk.download("stopwords")
# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\44758\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\44758\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\44758\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [7]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1') # avoids errors from special characters

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000) #why this is for now. just for speeding up development—l i can remove this line to use the full dataset.
print(data.shape) 
data.fillna("",inplace=True) 
data

(1000, 2)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
995,So what's the latest? It sounds contradictory ...,0
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1
997,Barb I will call to explain. Are you back in t...,0
998,Yang on travelNot free tonite.May work tomorrow,0


### Let's divide the training and test set into two partitions

In [8]:
# the usual traing 80% annd test 20%

from sklearn.model_selection import train_test_split

data_train, data_val, label_train, label_val = train_test_split(data, data['label'], test_size=0.2, random_state=42)

In [9]:
data_train

,text,label
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1
535,I have not been able to reach oscar this am. W...,0
695,; Huma Abedin B6I'm checking with Pat on the 5...,0
557,I can have it announced here on Monday - can't...,0
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1
...,...,...
106,7653 2612ADAMA IBRAHIM________________________...,1
270,What does that mean for our schedules?,0
860,"Dear Friend,My Compliment to you,I guess this ...",1
435,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,1


## Data Preprocessing

In [10]:
import string #used to handle removing punctuation
from nltk.corpus import stopwords # "the", "and", "is"


print(string.punctuation)
print(stopwords.words("english")[100:110]) #Displays 10 stopwords 


from nltk.stem.snowball import SnowballStemmer #form (e.g., "running" → "run").

snowball = SnowballStemmer('english') # Initializing the Snowball Stemmer

#Purpose of This Code
#Identifying punctuation to remove
#Listing stopwords to filter out
#Setting up a stemmer to normalize words

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [11]:
# Cleaning HTML
import re
import nltk

#remove inline JavaScript/CSS
def initial_clean_html(html_text):
   html_text = re.sub(r'<(script|style).*?>.*?</\1>', '', html_text, flags=re.DOTALL)

#remove html comments
   html_text = re.sub(r"<!--.*?-->", "", html_text, flags=re.DOTALL)#The dot (.) will match any character+ newline characters

   #Remove remaining HTML tags
   html_text = re.sub(r'<.*?>', '', html_text)
   return html_text  # Return the cleaned text

data_train['preprocess_text'] = data_train['text'].apply(initial_clean_html)
data_val['preprocess_text'] = data_val['text'].apply(initial_clean_html)

In [12]:
data_train

,text,label,preprocess_text
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE..."
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
...,...,...,...
106,7653 2612ADAMA IBRAHIM________________________...,1,7653 2612ADAMA IBRAHIM________________________...
270,What does that mean for our schedules?,0,What does that mean for our schedules?
860,"Dear Friend,My Compliment to you,I guess this ...",1,"Dear Friend,My Compliment to you,I guess this ..."
435,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,1,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [79]:
import re

def clean_html(html_text):
    
    # Remove all special characters (keep only letters and spaces)
    html_text = re.sub(r'[^a-zA-Z\s.]', '', html_text)

    # Remove numbers
    html_text = re.sub(r'\d+', '', html_text)   

    # Remove all single characters (e.g., 'a', 'b', 'c')
    html_text = re.sub(r'\b\w\b', '', html_text)

    # Remove single characters from the start
    html_text = re.sub(r'^\w\s+', '', html_text)

    # Substitute multiple spaces with single space
    html_text = re.sub(r'\s+', ' ', html_text).strip()

    # Remove prefixed 'b' 
    html_text = re.sub(r"^b['\"]", '', html_text)

    # Lowercase
    html_text = html_text.lower()

    return html_text

#  applying to my data
data_train['preprocess_text'] = data_train['text'].apply(clean_html)
data_val['preprocess_text'] = data_val['text'].apply(clean_html)


In [67]:
data_train

,text,label,preprocess_text
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regards mr nelson smith.kindly reply me on my ...
535,I have not been able to reach oscar this am. W...,0,have not been able to reach oscar this am. we ...
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin bim checking with pat on the will ...
557,I can have it announced here on Monday - can't...,0,can have it announced here on monday cant today
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank of africaagence san pedro bp san pedro co...
...,...,...,...
106,7653 2612ADAMA IBRAHIM________________________...,1,adama ibrahimtout savoir sur la scurit de votr...
270,What does that mean for our schedules?,0,what does that mean for our schedules
860,"Dear Friend,My Compliment to you,I guess this ...",1,dear friendmy compliment to youi guess this le...
435,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,1,dear presidentfdirectorc my name is mre michea...


## Now let's work on removing stopwords
Remove the stopwords.

In [68]:
stop_words = set(stopwords.words("english"))

stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [69]:
print(data_train.columns)  # Check if 'preprocessed_text' is a column in data_train
print(data_val.columns)    # Check if 'preprocessed_text' is a column in data_val


Index(['text', 'label', 'preprocess_text'], dtype='object')
Index(['text', 'label', 'preprocess_text'], dtype='object')


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources
nltk.download('punkt')  # For tokenizing words
nltk.download('wordnet')  # For WordNet lemmatization

# Function to perform lemmatization
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    words = word_tokenize(text)  # Tokenize the sentence into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return ' '.join(lemmatized_words)  # Join the words back into a sentence

# REMEBER TO X_train and X_test
# Preprocess the text data by applying lemmatization

# Apply lemmatization to the 'preprocessed_text' column in X_train
data_train['preprocess_text'] = data_train['preprocess_text'].apply(lemmatize_text)

# Apply lemmatization to the 'preprocessed_text' column in X_test
data_val['preprocess_text'] = data_val['preprocess_text'].apply(lemmatize_text)

# Check the result
print(data_train['preprocess_text'].head())
print(data_val['preprocess_text'].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\44758\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\44758\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


29     regard mr nelson smith.kindly reply me on my p...
535    have not been able to reach oscar this am . we...
695    huma abedin bim checking with pat on the will ...
557      can have it announced here on monday cant today
836    bank of africaagence san pedro bp san pedro co...
Name: preprocess_text, dtype: object
521    dear sirc wish you go through this offer to co...
737    to take your mind off the balkan for second se...
740                           pls keep the update coming
660    strongstrongchrist bethel hospitalbr rue abobo...
411    sbwhoeopfriday february amhre bravo brava issu...
Name: preprocess_text, dtype: object


In [81]:
print(data_train['preprocess_text'].head())

print(data_val['preprocess_text'].head())


29     regards mr nelson smith.kindly reply me on my ...
535    have not been able to reach oscar this am. we ...
695    huma abedin bim checking with pat on the will ...
557      can have it announced here on monday cant today
836    bank of africaagence san pedro bp san pedro co...
Name: preprocess_text, dtype: object
521    dear sirc wish you go through this offer to co...
737    to take your mind off the balkans for second s...
740                          pls keep the updates coming
660    strongstrongchrist bethel hospitalbr rue abobo...
411    sbwhoeopfriday february amhre bravo brava issu...
Name: preprocess_text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer for Bag of Words
vectorizer = CountVectorizer()

# top 10 words for ham and spam in matrix
bow_matrix = vectorizer.fit_transform(data_train['preprocess_text'])

matrix_ham = bow_matrix[data_train['label'] == 0].sum(axis=0).A1
matrix_spam = bow_matrix[data_train['label'] == 1].sum(axis=0).A1

# feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

top_10_ham = [feature_names[i] for i in matrix_ham.argsort()[-10:][::-1]]
top_10_spam = [feature_names[i] for i in matrix_spam.argsort()[-10:][::-1]]

print(top_10_ham)
print(top_10_spam)

print(matrix_ham)
print(matrix_spam)



['the', 'to', 'and', 'of', 'in', 'that', 'for', 'is', 'on', 'you']
['the', 'to', 'of', 'and', 'in', 'you', 'this', 'my', 'your', 'for']
[0 0 0 ... 0 1 0]
[1 2 3 ... 1 0 1]


## Extra features

In [123]:
# This code is adding extra features to the dataset to improve the performance 
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocess_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocess_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocess_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocess_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocess_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocess_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocess_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regards mr nelson smith.kindly reply me on my ...,1,0,87
535,I have not been able to reach oscar this am. W...,0,have not been able to reach oscar this am. we ...,1,0,94
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin bim checking with pat on the will ...,1,0,129
557,I can have it announced here on Monday - can't...,0,can have it announced here on monday cant today,1,0,47
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank of africaagence san pedro bp san pedro co...,1,1,1581


In [ ]:
# All rows in money_mark = 1,  suggests money-related symbols or words 
# (e.g., "euro," "dollar," "pound," "€," "$") are present in the preprocess_text.

# Only the last row has suspicious_words = 1, which means it contains words like "bank," "fund," or "transfer" from suspicious_w
# This is a strong indicator of spam

## How would work the Bag of Words with Count Vectorizer concept?

In [124]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
cv = CountVectorizer()

# Fit and transform the corpus into a BoW representation
X = cv.fit_transform(corpus).toarray()
 
# Print the results
print("BoW Matrix:\n", bow_array)

BoW Matrix:
 [[1 0 0 1 1 1 1 0]
 [0 1 1 0 1 0 0 1]]


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Load the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Step 2: Vectorize the entire dataset
# Fit on the training data and transform it
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['preprocess_text'])

# Transform the validation data using the same vectorizer
X_val_tfidf = tfidf_vectorizer.transform(data_val['preprocess_text'])

print("Vectorized Training Data:", X_train_tfidf.shape)
print("Vectorized Validation Data:", X_val_tfidf.shape)

Vectorized Training Data: (800, 17330)
Vectorized Validation Data: (200, 17330)


## And the Train a Classifier?

In [ ]:
# Step 1: Combine TF-IDF vectors with additional features
from scipy.sparse import hstack
X_train_combined = hstack((X_train_tfidf, data_train[['money_mark', 'suspicious_words', 'text_len']]))

# Step 2: Train the classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=42)

classifier.fit(X_train_combined, data_train['label'])


RandomForestClassifier(random_state=42)

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code